In [1]:
import copy, sys, pandas as pd
import random
import time
import numpy as np
import subprocess

sys.path.append("../../")
sys.path.append("../../inferring")
sys.path.append("../../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS
import utils.advice_systems.SRSconv_with_common as SRS_common

reload(SRS_common)

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.advice_systems.SRSconv_with_common import SRSconv_with_common
from inferring.InferringDFA import InferringDFA

In [2]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [5]:
class Record:
    def __init__(self,
                 dfa_sz,
                 common,
                 lstar,
                 lstar_as, 
                 target=None):
        self.dfa_sz = dfa_sz
        self.common = common
        self.lstar = lstar
        self.lstar_as = lstar_as
        self.target = target

    def print_record(self):
        print(
            f"|dfa_sz| = {self.dfa_sz}, common letters = {int(self.common*100)}% eq = {self.lstar[1]}, eq_with_advice = {self.lstar_as[1]}"
        )

    def get_complete_description(self):
        return self.target.print_complete_description()

    def print_target(self):
        print(self.target.print_complete_description())

In [18]:
import string

max_number_of_states = 30
common_letters = [4, 3, 2, 1, 0]
unique_letters = [1, 1, 2, 2, 3]
number_of_iter_per_case = 5
number_of_iteration = number_of_iter_per_case * len(common_letters)

results = []
i = 0

_ = subprocess.run(["cd ../../../learnlib/examples ; mvn clean install; cd ../../magisterka/test_algorithm/Lstar"], shell=True, capture_output=True, text=True)
while i < number_of_iteration:
    random.seed(i)
    k = common_letters[i % len(common_letters)]  # number of common letters
    l = unique_letters[i % len(unique_letters)]  # number of unique letters
    input_signs = [a for a in string.ascii_lowercase[:l + k + l]]
    i += 1
    # Create two random DFAs and run learning process to minimize dfa1 and dfa2
    d1 = DFA()
    d2 = DFA()
    while d1.Q < 2:
        dfa1 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs[:k + l],
        )
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    while d2.Q < 2:
        dfa2 = DFA()
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs[-(k + l):],
        )
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    conv_d = DFA()
    conv_d.create_convolution_with_common_letters(d1, d2)
    # conv_d.print_dfa()

    f = open(
        "../../../learnlib/examples/src/main/java/de/learnlib/example/DfaEx" +
        ".txt", "w")
    f.write(conv_d.print_complete_description())
    f.close()

    # Learn conv_d without advice
    lstar_output = subprocess.run([
        "cd ../../../learnlib/examples ; mvn exec:java -q -Dexec.mainClass=\"de.learnlib.example.LstarExample1\" ; cd ../../magisterka/test_algorithm/Lstar"
    ],
                                  shell=True,
                                  capture_output=True,
                                  text=True)
    # Learn conv_d with advice
    lstar_as_output = subprocess.run([
        "cd ../../../learnlib/examples ; mvn exec:java -q -Dexec.mainClass=\"de.learnlib.example.LstarExample_conv_cl_withAS\" ; cd ../../magisterka/test_algorithm/Lstar"
    ],
                                     shell=True,
                                     capture_output=True,
                                     text=True)

    def getNumberofEQ(s):
        for l in s.splitlines():
            if l.startswith("Learning rounds"):
                return int(l.split()[-1])

    results.append(
        Record(dfa_sz=conv_d.Q,
               lstar=['?', getNumberofEQ(lstar_output.stdout)],
               lstar_as=['?', getNumberofEQ(lstar_as_output.stdout)],
               common=k / (l + k),
               target=conv_d))
    print(f"iter {i} done")

iter 1 done
iter 2 done
iter 3 done
iter 4 done
iter 5 done
iter 6 done
iter 7 done
iter 8 done
iter 9 done
iter 10 done
iter 11 done
iter 12 done
iter 13 done
iter 14 done
iter 15 done
iter 16 done
iter 17 done
iter 18 done
iter 19 done
iter 20 done
iter 21 done
iter 22 done
iter 23 done
iter 24 done
iter 25 done


In [19]:
results.sort(key=lambda x: (x.common, x.dfa_sz))
print(f"number of tests: {len(results)}")

number of tests: 25


In [21]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_eq_red = df[(('Reduction', 'EQ'))].mean()
    max_eq_red = df[(('Reduction', 'EQ'))].max()
    min_eq_red = df[(('Reduction', 'EQ'))].min()
    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row,
               params=[(0, "Max reduction"), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row,
               params=[(0, "Min reduction"), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row,
               params=[(0, "Average reduction"), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)


    for i in range(len(common_letters)):
        df = pd.concat([df.iloc[:(i+1)*number_of_iter_per_case + i], empty_row, df.iloc[(i+1)*number_of_iter_per_case + i:]]).reset_index(drop=True)

    df[('Reduction', 'EQ')] = df[(
        'Reduction',
        'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    df[('Common letters')] = df[('Common letters')].apply(
        lambda x: f'{x}%' if isinstance(x, int) else x)

    return df

In [22]:
lstar_eqs = [r.lstar[1] for r in results]
lstar_as_eqs = [r.lstar_as[1] for r in results]

red = (
    (np.array(lstar_eqs) - np.array(lstar_as_eqs)) / np.array(lstar_eqs)) * 100
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Common letters', ''), ('L*', 'EQ'), ('L* with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.dfa_sz for r in results],
    columns[1]: [int(r.common*100) for r in results],
    columns[2]: lstar_eqs,
    columns[3]: lstar_as_eqs,
    columns[4]: [int(r) for r in red]
}

df = create_table(columns=columns, data=data)
display(df)

,Target language,Common letters,L*,L* with advice,Reduction
,"conv(DFA1, DFA2)",,EQ,EQ,EQ
0,260,0%,64,6,90%
1,330,0%,106,4,96%
2,396,0%,111,2,98%
3,450,0%,134,2,98%
4,459,0%,92,1,98%
5,,,,,
6,288,33%,47,3,93%
7,360,33%,64,3,95%
8,441,33%,117,4,96%
